In [1]:
!nvidia-smi

Thu Feb 10 04:20:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    52W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !pip install wandb
# !pip install opencv-python==4.1.2.30
# !pip install albumentations==0.5.2
# !pip install timm

In [4]:
import os
import random
import math

from glob import glob
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.loss import _Loss
from torch.optim.lr_scheduler import _LRScheduler

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import timm

In [5]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

# Dataset

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, transforms, root='./train/train'):
        self.data = data
        self.transforms = transforms
        self.root = root
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        src, trg = self.data[idx]
        img = cv2.imread(self.root + '/' + src)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img, trg
    

In [7]:
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(),
        A.Normalize(mean=[0.59534838, 0.5949003 , 0.59472117], std=[0.29705278, 0.29707744, 0.29702731]),
        ToTensorV2()
    ])

def get_valid_transforms():
    return A.Compose([
        A.Normalize(mean=[0.59534838, 0.5949003 , 0.59472117], std=[0.29705278, 0.29707744, 0.29702731]),
        ToTensorV2()
    ])

In [8]:
def split_stratified_shuffle_split(fold, n_split, seed):
    skf = StratifiedShuffleSplit(n_splits=n_split, random_state=seed)
    for idx, i in enumerate(skf.split(df['file_name'], df['COVID'])):
        if idx == fold:
            train_data = df.values[i[0]]

            valid_data = df.values[i[1]]
    return train_data, valid_data

# Model

In [9]:
timm.list_models('*')

m = timm.create_model('seresnet18', pretrained=False)

In [10]:
m.fc.in_features

512

In [11]:
class MyModel(nn.Module):
    def __init__(self, n_classes, model_name):
        super(MyModel, self).__init__()
        self.feature = timm.create_model(model_name, pretrained=False)
        # convnext 계열
        # self.feature.head.fc = nn.Linear(in_features=2048, out_features=n_classes, bias=True) 

        # resnet 계열
        self.out_features = self.feature.fc.in_features
        self.feature.fc = nn.Linear(in_features=self.out_features, out_features=n_classes, bias=True) 
    
    def forward(self, x):
        x = self.feature(x)
        return x

# Initialization

In [12]:
def init_weight(model, kind='xavier'):
    for name, i in model.named_parameters():
        if kind == 'xavier':
            if i.dim() < 2:
                continue
            if 'weight' in name:
                init.xavier_normal_(i, gain=1.0)
            elif 'bias' in name:
                init.xavier_uniform_(i, gain=1.0)
            else:
                pass
        elif kind == 'kaiming':
            if i.dim() < 2:
                continue
            if 'weight' in name:
                init.kaiming_normal_(i, gain=1.0)
            elif 'bias' in name:
                init.kaiming_uniform_(i, gain=1.0)
            else:
                pass

# Loss

In [13]:
class MyLoss(_Loss):
    def __init__(self):
        super(MyLoss, self).__init__()
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, preds, trg):
        return self.loss(preds, trg)

# Train Functions

In [14]:
def train_one_epoch(model, criterion, train_loader, scheduler, device):
    model.train()
    
    losses = 0
    match = 0
    for idx, (src, trg) in tqdm(enumerate(train_loader)):
        src, trg = src.to(device).float(), trg.to(device).long()
        
        outs = model(src)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, trg)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        losses += loss.item()
        match += (preds == trg).sum().item()
    return losses, match
        
def valid_one_epoch(model, criterion, valid_loader, device):
    model.eval()
    print("validation")
    
    losses = 0
    match = 0
    with torch.no_grad():
        for idx, (src, trg) in tqdm(enumerate(valid_loader)):
            src, trg = src.to(device).float(), trg.to(device).long()
            
            outs = model(src)
            preds = torch.argmax(outs, dim=-1)
            
            loss = criterion(outs, trg)
            
            losses += loss.item()
            match += (preds == trg).sum().item()
    return losses, match

# Early Stop

In [15]:
class EarlyStopper():

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.best_acc = 0
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, score: float)-> None:

        if self.best_acc == 0:
            self.best_acc = score
            return None

        elif score < self.best_acc:
            self.patience_counter += 1
            self.save_model = False
            if self.patience_counter == self.patience:
                self.stop = True
                
        elif score >= self.best_acc:
            self.patience_counter = 0
            self.save_model = True
            self.best_acc = score
            print('best score :', self.best_acc)
        print("best_acc", self.best_acc)

# Scheduler

In [16]:
class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """

    def __init__(self,
                 optimizer: torch.optim.Optimizer,
                 first_cycle_steps: int,
                 cycle_mult: float = 1.,
                 max_lr: float = 0.1,
                 min_lr: float = 0.001,
                 warmup_steps: int = 0,
                 gamma: float = 1.,
                 last_epoch: int = -1
                 ):
        assert warmup_steps < first_cycle_steps

        self.first_cycle_steps = first_cycle_steps  # first cycle step size
        self.cycle_mult = cycle_mult  # cycle steps magnification
        self.base_max_lr = max_lr  # first max learning rate
        self.max_lr = max_lr  # max learning rate in the current cycle
        self.min_lr = min_lr  # min learning rate
        self.warmup_steps = warmup_steps  # warmup step size
        self.gamma = gamma  # decrease rate of max learning rate by cycle

        self.cur_cycle_steps = first_cycle_steps  # first cycle step size
        self.cycle = 0  # cycle count
        self.step_in_cycle = last_epoch  # step size of the current cycle

        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)

        # set learning rate min_lr
        self.init_lr()

    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)

    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr) * self.step_in_cycle / self.warmup_steps + base_lr for base_lr in
                    self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle - self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int(
                    (self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(
                        self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch

        self.max_lr = self.base_max_lr * (self.gamma ** self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

# Train

In [17]:
!wandb login

wandb: Currently logged in as: yeardream7 (use `wandb login --relogin` to force relogin)


In [18]:
import wandb

In [19]:
CFG = {
    'seed':2022,
    'fold':3, # 학습시킬 fold
    'n_split':5, # fold 개수
    'batch_size': 16,
    'num_classes': 2,
    'epoch': 100,
    'model_name': "resnext50_32x4d",
    'initialization': "xavier" # kaiming
}

root_dir = '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification'

wandb.config = CFG
run = wandb.init(project=f"{CFG['model_name']}", settings=wandb.Settings(start_method="thread"), name=f"xavier_fold{CFG['fold']}")

wandb: Currently logged in as: yeardream7 (use `wandb login --relogin` to force relogin)


In [21]:
# seed 고정
seed_everything(CFG['seed'])
# device 설정
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Data 설정
df = pd.read_csv(root_dir + '/' + 'train/train.csv')
train_data, valid_data = split_stratified_shuffle_split(CFG['fold'], CFG['n_split'], CFG['seed'])

train_transforms = get_train_transforms() # augmentation
valid_transforms = get_valid_transforms()

train_dataset = MyDataset(train_data, train_transforms, root_dir + '/' + 'train/train/')
valid_dataset = MyDataset(valid_data, valid_transforms, root_dir + '/' + 'train/train/')

train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG['batch_size'],
    num_workers=0,
    shuffle=True,
    pin_memory=use_cuda,
    drop_last=True,)

valid_loader = DataLoader(
    valid_dataset, 
    batch_size=CFG['batch_size'],
    num_workers=0,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False)
# 모델, Loss, scheduler, optimizer 설정
model = MyModel(CFG['num_classes'], CFG['model_name']).to(device)
init_weight(model, kind=CFG['initialization'])

criterion = MyLoss()

cosine_annealing_scheduler_arg = dict(
    first_cycle_steps=len(train_dataset)//CFG['batch_size'] * 10,
    cycle_mult=1.0,
    max_lr=1e-03,
    min_lr=1e-07,
    warmup_steps=len(train_dataset)//CFG['batch_size'] * 2,
    gamma=0.9
)

optimizer = optim.Adam(model.parameters(), lr=0.000, weight_decay=0)
scheduler = CosineAnnealingWarmupRestarts(optimizer, **cosine_annealing_scheduler_arg)

early_stopper = EarlyStopper(patience=10)

In [ ]:
print('Start Training!')
for i in range(CFG['epoch']):
    print(f"Epoch :", i)
    train_losses, train_match = train_one_epoch(model, criterion, train_loader, scheduler, device)
    print("Train loss, score:", train_losses / len(train_loader), train_match / len(train_loader.dataset))
    valid_losses, valid_match = valid_one_epoch(model, criterion, valid_loader, device)
    print("Valid loss, score:", valid_losses / len(valid_loader), valid_match / len(valid_loader.dataset))
    early_stopper.check_early_stopping(valid_match / len(valid_loader.dataset))

    wandb_dict = {
        'train loss': train_losses / len(train_loader),
        'train score': train_match / len(train_loader.dataset),
        'valid loss': valid_losses / len(valid_loader),
        'valid score': valid_match / len(valid_loader.dataset),
        'learning rate': scheduler.get_lr()[0]
    }

    wandb.log(wandb_dict)

    print(scheduler.get_lr())

    if early_stopper.save_model == True:
        dic = {
            'model':model.state_dict(),
            'optimizer':optimizer.state_dict(),
            'scheduler':scheduler.state_dict(),
        }
        torch.save(dic, root_dir + f"/pth/{CFG['fold']}_best_{CFG['model_name']}.pth")
        print('save_model')

    if early_stopper.stop:
        break

os.rename(
    root_dir + f"/pth/{CFG['fold']}_best_{CFG['model_name']}.pth", 
    root_dir + f"/pth/{CFG['fold']}_{early_stopper.best_acc:.4f}_{CFG['model_name']}.pth"
    )

Start Training!
Epoch : 0


0it [00:00, ?it/s]

Train loss, score: 0.6769414155019654 0.6282271944922547
validation


0it [00:00, ?it/s]

Valid loss, score: 0.5613742291927337 0.7384615384615385
[0.0005000499999999999]
Epoch : 1


...

0it [00:00, ?it/s]

Train loss, score: 0.3508577953196234 0.8623063683304647
validation


0it [00:00, ?it/s]

Valid loss, score: 0.6380409199744463 0.7076923076923077
best_acc 0.7846153846153846
[0.0005600310242291876]
Epoch : 44


0it [00:00, ?it/s]

# Ensemble

In [22]:
model_paths = [
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/0_0.8923_resnext50_32x4d.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/1_0.7692_resnext50_32x4d.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/2_0.7846_resnext50_32x4d.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/3_0.7692_resnext50_32x4d.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/4_0.7692_resnext50_32x4d.pth'
]

In [27]:
model = MyModel(CFG['num_classes'], CFG['model_name']).to(device)
df_test = pd.read_csv(root_dir + '/' + 'test/sample_submission.csv')

test_transforms = get_valid_transforms()
test_dataset = MyDataset(df_test.values, test_transforms, root_dir + '/' + 'test/test/')

test_loader = DataLoader(
    test_dataset, 
    batch_size=CFG['batch_size'],
    num_workers=0,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False)

In [32]:
result = None

with torch.no_grad():
    for i in tqdm(model_paths):

        checkpoint = torch.load(i)
        model.load_state_dict(checkpoint['model'])
        model.eval()

        for j in range(2): 
            tmp = []
            for idx, (src, trg) in tqdm(enumerate(test_loader)):
                
                src, trg = src.to(device).float(), trg.to(device).long()
                if j == 0:
                    src = torch.flip(src, [-1])
                outs = model(src)
                tmp.extend(outs.detach().cpu().numpy().tolist())
            if result is None:
                result = np.array(tmp)
            else:
                result += np.array(tmp)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [33]:
preds = np.argmax(result, axis=-1) # tta
preds 

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0])

In [34]:
df_test['COVID'] = preds

In [35]:
df_test.to_csv('/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/submission/submission_resnext_tta.csv', index=False)